## Environment setup

### Library preparation

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [75]:
import time, os, io
from PIL import ImageGrab
from IPython.display import Image, display
import clipboard
from scipy import stats
from si_prefix import si_format
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
from io import StringIO 
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
plt.close('all')

import sys 
sys.path.append('..')
from meas_util import *

import qcodes as qc
from qcodes import Station, load_by_run_spec, load_by_guid
from qcodes.instrument.base import Instrument
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.database import initialise_database
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id, get_data_by_id, plot_dataset
from qcodes.dataset.data_set import load_by_id
# from qcodes.dataset.data_export import get_shaped_data_by_runidb

In [3]:
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.tektronix.AWG3252_Isrc import AWG3252_Isrc
from qcodes.instrument_drivers.HP.HP34401 import HP34401
from qcodes.instrument_drivers.HP.HP34401_DL1201 import HP34401_DL1201
from qcodes.instrument_drivers.Keithley.Keithley_2600_channels import *
from qcodes.instrument_drivers.tektronix.AWG5204_Isrc import AWG5204_Isrc

from qcodes.instrument_drivers.Keithley.Keithley_6220 import Keithley_6220

### Instrument nicknames and QC database initialization

In [4]:
try:
   Instrument.close_all()
except KeyError:
   pass    
except NameError:
   pass 

K2602 =  Keithley_2600(name = 'K2602', address = "GPIB::26::INSTR" )
Flux_chA = K2602.smua.curr
Flux_chB = K2602.smub.curr
Bfield =  Flux_chA
Bfield.step = 10e-6
Bfield.inter_delay = .1

LS370 = Model_372(name = 'LS370 input Ch', address = 'GPIB::12::INSTR', terminator='\n')
T8 = LS370.ch08.temperature

htr = LS370htr()
htr.Tset(0.030)
htr.PIDset(30,5,1)
htr.HTRset('off')

# Isrc = AWG5204_Isrc('gen', 'TCPIP0::AWG52000-XXXX::inst0::INSTR', ch = 1, R_bias = 1e9, Attn = 1, timeout = 20)
Isrc = Keithley_6220(name = 'K6220', address = "GPIB::13::INSTR" )
# Isrc = AWG3252_Isrc('gen', 'TCPIP0::192.168.13.32::inst0::INSTR',  R_bias = 1e9, Attn = 1)
Isrc.init()

Vmeter = HP34401_DL1201('meter', 'GPIB0::8::INSTR', Gain = 100)
Vmeter.init('fast 6')

station = qc.Station(Isrc, Vmeter)

print ('Instrument initialization completed.')

Connected to: Keithley Instruments Inc. 2602A (serial:1272049, firmware:2.1.5) in 0.49s
Connected to: LSCI 370 (serial:370665, firmware:04102008) in 0.11s
initialization \\JOSH-PC\Gersh_Labview\DC measurement\LSCI 370 PYTHON_conf.vi
Instrument initialization completed.


### Database ini

In [5]:
SAMPLE = '79N5'
qc.config["core"]["db_location"] = r'\\JOSH-PC\OurData_OneDrive\Experiments_{}.db'.format(SAMPLE)

## Scripts

### Utilities

In [6]:
def setup():
    Isrc.I.set(0)

def cleanup():
    Isrc.I.set(0) 
    Bfield.set(0)
    
def status_now():
    print(Isrc)
    print(Vmeter)
    print('B = {}A'.format(si_format(Bfield.get())))
    print('T = {}K'.format(si_format(T8.get())))
    print('{}\n{}'.format(htr.HTRget(), htr.PWRget()))

In [7]:
def meas_Voffset(i):
    V_off = 0
    N = 10
    Isrc.I.set(i)


    for j in range(N):
#         time.sleep(.1)
        V_off += Vmeter.V.get()
    return V_off/N

In [8]:
def cos_to_B(cos, ZF = .004e-3, FF = .232e-3):

    return np.arccos(cos)*(2* (FF - ZF)/np.pi + ZF  )

In [9]:
def T_scan(T_list):

    tolerT8 = 0.02
    chkrepeat = 20
    chkperiod_sec = 2

    Tnow_alllst = []
    timer_lst = []

    htr.HTRset('10mA')

    for setT8 in setT8_lst:
    #     htr.setpoint(setT8)
        htr.Tset(setT8)
        print('ramping T8 to {}K...'.format(si_format(setT8)))
        if setT8 < 60e-3:
            time.sleep(300)
        else:
            time.sleep(30)
        Tnow_lst = []

        t0 = time.time()
        count_T = 0
        while count_T < chkrepeat:
            T_now = T8.get()
            if (1-tolerT8)*setT8 <= T_now <= (1+tolerT8)*setT8 :
                count_T +=1
                Tnow_lst.append(T_now)
                time.sleep(chkperiod_sec)
            elif count_T >= 1 :
                count_T -=1
        t1 = time.time()

        timer_lst.append(t1-t0)
        Tnow_alllst.append(Tnow_lst)






### IVCs

In [82]:
def IVC_udu (amp, stp, Gain, R_bias, Attn, 
             tfsstb = 0.1, label = '', NEGATIVE = True):
    def ud_list(amp, stp):
    
        n  =  np.round(amp/stp)
        
        u1 = np.linspace (0, amp, n + 1)
        d = np.linspace (amp, 0, n +1)
        
    
        return np.concatenate ([u1, d])
    
    i0 = 20e-12

    Isrc.I.set(i0)
    time.sleep(1)
    Isrc.I.set(-i0)
    time.sleep(1)
   
    Voff =  meas_Voffset(0)
    
    meas = set_meas(Vmeter.V, Isrc.I, setup = setup, cleanup = cleanup)

    Vmeter.set_Gain ( Gain )
    Isrc.set_R_Attn ( R_bias, Attn )
    
    if NEGATIVE:
        i_list = udu_list(amp, stp)
    else:
        i_list = ud_list(amp, stp)
    
    ti_list = tqdm_notebook(i_list)
    if label == '':
        label = 'IVc @T={}K, B={}A'.format(si_format(T8.get()), si_format(Bfield.get(), precision = 3))
        

    name_exp(sample = SAMPLE, exp_type = label )
    with meas.run() as datasaver:
        for i in ti_list:
            
            Isrc.I.set(i)
            time.sleep(tfsstb)
            V = Vmeter.V.get()

            res = [( Isrc.I, i - 0*22.5e-3/R_bias*Attn ), ( Vmeter.V, V - Voff  )]
            datasaver.add_result(*res) 

    cleanup()
    return datasaver.run_id 

In [11]:
def Ic_meas (amp, stp, thr, Gain, R_bias, Attn, tfsstb = 0.1):

    meas = set_meas(Vmeter.V, Isrc.I, setup = setup, cleanup = cleanup)
    Vmeter.set_Gain (Gain) 
    Isrc.set_R_Attn (R_bias, Attn)
    
    i_list_u = np.linspace(0,  amp,  round (amp/stp) + 1)
    ti_list_u = tqdm_notebook(i_list_u)
    i_list_d = np.linspace(0, -amp,  round (amp/stp) + 1)    
    ti_list_d = tqdm_notebook(i_list_d)
    ti_lists = [ ti_list_u ,  ti_list_d]
    
    name_exp(sample = SAMPLE, exp_type = 'sc branch @T={}K, B={}A'.format(si_format(T8.get()), si_format(B, precision = 3)))
    with meas.run() as datasaver:
        for ti_list in ti_lists:
            
            Isrc.I.set(0)
            V_ = Vmeter.V.get()            
            time.sleep(1)
            
            for i in ti_list:
            
                Isrc.I.set(i)
                time.sleep(tfsstb)

                V = Vmeter.V.get()
                dV = V - V_
                V_ = V 
                
                if (abs(dV) > thr)  or (abs(V) > 4e-3 ) : #
                    break

                res = [( Isrc.I, i - 22.5e-3/R_bias*Attn ), ( Vmeter.V, V  )]
                datasaver.add_result(*res) 
    cleanup()
    return datasaver.run_id 

In [12]:
def IVcB_meas (i_list, B_list, thr=1e-3, Gain=100, R_bias=1e9, Attn=1, tfsstb = 0.1):

    Vmeter.set_Gain (Gain = Gain) 
    Isrc.set_R_Attn (R_bias = R_bias, Attn=Attn)

    i_lists = [i_list]
    
    meas = set_meas(Vmeter.V, Isrc.I,setup = setup, cleanup = cleanup)
    tB_list = tqdm_notebook(B_list)
    for B in tB_list:

        name_exp(sample = SAMPLE, exp_type = 'full IVc(B) @T={}K, B={}A'.format(si_format(T8.get()), si_format(B, precision =3)))
        with meas.run() as datasaver:

            Bfield.set(B)
            for i_list in i_lists:

                Isrc.I.set(0)
                V_ = Vmeter.V.get()
                time.sleep(1)

                for i in i_list:

                    Isrc.I.set(i)
                    time.sleep(tfsstb)

                    V = Vmeter.V.get()
                    dV = V - V_
                    V_ = V

                    if  (abs(V) > 6.4e-3) : #(abs(dV) > thr)  or
                        break

                    res = [( Isrc.I, i - 22.5e-3/R_bias*Attn ), ( Vmeter.V, V  )]
                    datasaver.add_result(*res) 
    cleanup()
    return datasaver.run_id 

### Histograms

In [86]:
def meas_hist(N, istep =0.1e-9,t_offset = 0, Vthr = 200e-6 ):

    Isws = []

    i = 0
#    istep = .2e-9
    i0 = 10*istep

    Isrc.I.set(i0)
    time.sleep(1)
    Isrc.I.set(-i0)
    time.sleep(1)
    V_off = meas_Voffset(0)

    t0 = time.time()    
    for n in tqdm_notebook(range(N)):

#         print(1e3*(time.time()-t0))
        V = V_off

        while (V - V_off) < Vthr:
            
            i += istep
            time.sleep(t_offset)
            Isrc.I.set(i)
            V = Vmeter.V.get()
            
        Isws.append(i)
        #print(i)

        Isrc.I.set(-i0)
        time.sleep(.1)
        i = np.mean(Isws)*0.7
#         Isrc.I.set(i)
        time.sleep(0.1)

    return Isws

### Data proc

In [14]:
def extract_Isw_by_id (idx):
    
    alldata = get_data_by_id(idx)
    
    Is = alldata[0][0]['data']
    Vs = alldata[0][1]['data']

        
    return extract_Isw (Is,Vs)

def extract_Isw (Is,Vs):
    
        if len( Is )== 0 or len( Vs )== 0 :
            Isw, R0 = np.nan, np.nan
            return Isw, R0
        
        Isw = abs(np.max(Is)) 
        

        
        return Isw

In [15]:
def extract_Isw_R0 (Is,Vs):

    Isw = (np.max(Is) - np.min(Is) ) /2

    order = Is.argsort()        
    Is, Vs = Is[order], Vs[order]
    n = len(Is)
    n_min, n_max = np.int(n/3), np.int(2*n/3)        
    n_sl = slice(n_min, n_max)

    R0 = np.mean(np.diff(Vs[n_sl])) / np.mean(np.diff(Is[n_sl]))        
    return Isw, R0

def extract_Isw_R0_by_id (idx):
    
    alldata = get_data_by_id(idx)
    
    Is = alldata[0][0]['data']
    Vs = alldata[0][1]['data']
        
    return extract_Isw_R0 (Is,Vs)

In [16]:
def xy_by_id(idx):
    alldata = get_data_by_id(idx)
    
    x = alldata[0][0]['data']
    y = alldata[0][1]['data']
    
    return x,y

In [17]:
def plot_hist_by_id(idx, ax = None):
    _, Isws = xy_by_id(idx)
    if ax is None:

        fig, ax = plt.subplots()

    ax.hist(Isws, bins = 201, cumulative = True )
    return Isws

In [18]:
def save_hist(Isws, label = ''):     
    N_try = qc.Parameter('try')    

    meas = set_meas( Isrc.I, N_try, setup = setup, cleanup = cleanup)
    name = '{:} @T ={:2.1f}mK'.format(label, T8.get()/1e-3)
    name_exp(sample = SAMPLE, exp_type = name )  
    with meas.run() as datasaver: 
        for j, I in enumerate(Isws):
            res = [ ( N_try, j  ) ,( Isrc.I, I ),]

            datasaver.add_result(*res)
            
    print('Run # {:1.0f}, {:} saved'.format(datasaver.run_id, name))
    return(datasaver.run_id )

## Measurement

#### Constants

In [19]:
ZF =  0.000e-3
FF =  0.227e-3
# dF = 0.250e-3
# midF = 0.154e-3




#### Iscr choise

In [99]:
Isrc.close()
# Isrc = AWG5204_Isrc('gen', 'TCPIP0::AWG52000-XXXX::inst0::INSTR', ch = 1, R_bias = 1e9, Attn = 1, timeout = 20)
Isrc = Keithley_6220(name = 'K6220', address = "GPIB::13::INSTR" )
# Isrc = AWG3252_Isrc('gen', 'TCPIP0::192.168.13.32::inst0::INSTR',  R_bias = 100e6, Attn = 1)
Isrc.init()

In [95]:
Isrc.I.set(0.04e-9)


In [104]:
Isrc.write_raw('SOUR:CURR:RANG 1e-8')
Isrc.I.set(0.0e-9)

#### Vmeter regime

In [44]:
Bfield.set(0)

In [27]:
Vmeter.V.get()

1.835723e-05

In [115]:
Vmeter.init('fast 6')

### IVCs

In [38]:
IVC_udu (amp= 20e-9, stp= .2e-9, 
             Gain=100, R_bias=101.2e6, Attn=1, tfsstb = 0.1,
             label = 'Full IVC' )

Starting experimental run with id: 11



11

In [ ]:
IVcB_ada(npts_B = 51, thr=250e-6, Gain=100, R_bias=11.2e6, Attn=1, tfsstb = 0.1)

In [61]:
np.cos(1/2.7*np.pi)

0.3960797660391569

In [120]:
# for B in np.linspace(160e-6, 280e-6, 21) :

# for B in [224e-6,230e-6]:
for cos in [0.15]*10 : 
# for cos in tqdm_notebook(np.linspace(1,0,51 )):
    B = cos_to_B(cos, FF = 210e-6, ZF = 0e-6)
    print(B)
    Bfield.set(B)
    IVC_udu (amp=.17e-9, stp= 0.002e-9, NEGATIVE = False, 
             Gain=100, R_bias=101.2e6, Attn=1, tfsstb = .05,
             label = 'cos = {:1.2f} '.format(cos) )

Bfield.set(0)

0.00018987050469641652


C:\Users\KelvinOX25\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  import sys
C:\Users\KelvinOX25\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  


Starting experimental run with id: 176

0.00018987050469641652


Starting experimental run with id: 177

0.00018987050469641652


Starting experimental run with id: 178

0.00018987050469641652


...

Starting experimental run with id: 181

0.00018987050469641652


Starting experimental run with id: 182

0.00018987050469641652


Starting experimental run with id: 183

0.00018987050469641652


Starting experimental run with id: 184

0.00018987050469641652


Starting experimental run with id: 185



In [ ]:
Vmeter.init('fast 4')
Blst = cos_to_B(np.linspace(0.4,-0.4,9))
idlst = []
for B in Blst:
    Bfield.set(B)
    idx = IVC_udu (amp=40e-9, stp=0.2e-9, Gain=100, R_bias=11.2e6, Attn=1, tfsstb = 0.1)
    idlst.append(idx)
Bfield.set(0)
print(idlst)

### Histograms

In [ ]:
B = cos_to_B(.25)
Bfield.set(B)
Isws =  meas_hist(1000)
save_hist(Isws)

In [ ]:
meas = set_meas(Vmeter.V, Isrc.I, setup = setup, cleanup = cleanup)

name_exp(sample = SAMPLE, exp_type = 'IVC for hist @T ={:2.1f}mK'.format(T8.get()/1e-3) )  
with meas.run() as datasaver:

    Is, Vs = fast_IVc_adaptive(Gain=100, R_bias=10e6, Attn=1, find_istep = istep_coarse, 
                               keepgoing =  if_Vth, direction = 'p', record = 'u')
    
print(Is)
print(extract_Isw (Is, Vs))


In [103]:
# for cos in np.linspace(.1,.25, 7):
for cos in [1]:
    B = cos_to_B(cos)
    Bfield.set(B)
#     IVC_udu (amp=20e-9, stp=.2e-9, Gain=100, R_bias=11.2e6, Attn=1, tfsstb = 0.1)
#     Bfield.set(B)
    Isws =  meas_hist(1000, istep =0.02e-9)
    save_hist(Isws, 'keith, cos = {:1.2f}'.format(cos))
Bfield.set(0)


Starting experimental run with id: 134
Run # 134, keith, cos = 1.00 @T =43.4mK saved


In [ ]:
Vmeter.init('fast 4')
for t in range(0,35,5):
    save_hist(meas_hist(N=100,t_offset = t/(150e-9*.9/.2e-9)), 'KE delay {}s/it'.format(t))

In [87]:
data = []
for i in range(2,33):
    data.append(i)
print(data)

# just FYI: data = [i for i in range(2,23)] does the same

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
